# Classical Shadows

The output of a quantum computer is a histogram of measurements, corresponding to the different outcomes observed, usually expressed as bitstrings. The cost and duration of a quantum experiment is roughly linear with the number of shots used to build such histograms, which also correlates with the accuracy of the results. The emergent method of classical shadows ([Nat. Phys. 16, 1050–1057 (2020)](https://arxiv.org/abs/2002.08953)) have been developed to mitigate the measurement overhead by offloading quantum tasks to the pre- and post-processing steps. This prediction protocol exhibits logarithmic scaling with the number of shots to evaluate observables within a wanted accuracy. 

Tangelo users can leverage this protocol by performing the relevant pre- and post-processing functions, as introduced in this notebook.

## System

To demonstrate the classical shadow capabilities, we first define a molecular system composed of two hydrogen atoms in the 3-21G basis. When translated to a qubit language, it  

In [7]:
from tangelo.molecule_library import mol_H2_321g
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping
from tangelo.toolboxes.operators import count_qubits

encoding = "scBK"
up_then_down = True

n_spinorbs = mol_H2_321g.n_active_sos
n_elecs = mol_H2_321g.n_active_electrons

qubit_ham = fermion_to_qubit_mapping(mol_H2_321g.fermionic_hamiltonian, 
                                     encoding, 
                                     n_spinorbitals=n_spinorbs, 
                                     n_electrons=n_elecs, 
                                     up_then_down=up_then_down)

print(f"Number of qubits to describe the system: {count_qubits(qubit_ham)}")

Number of qubits to describe the system: 6


For this example, the qubit coupled-cluster (QCC) ansatz have been chosen (Journal of Chemical Theory and Computation 2018 14 (12), 6317-6326. 10.1021/acs.jctc.8b00932). The QCC ansatz operator $\hat{U}(\tau)$ is specified according to the equation
$$
\hat{U}(\tau) = \prod_k^{n_g} \exp{-\frac{i\tau_k \hat{P}_k}{2}}
$$

While the depth of the circuit rapidly increases as the size of the molecule increases, the QCC ansatz admits a low-depth quantum circuit compared to a widely used unitary coupled-cluster single and double ansatz. 

In [8]:
from tangelo.toolboxes.ansatz_generator.qcc import QCC

ansatz = QCC(mol_H2_321g, mapping=encoding, up_then_down=up_then_down)
ansatz.build_circuit()

print(f"Number of gates in the circuit: {ansatz.circuit.size}")

Number of gates in the circuit: 79


The Variational Quantum Eigensolver (VQE) has been introduced in another notebook as a hybrid quantum–classical algorithm for simulating quantum systems. In the next cell, we focus on VQE within the context of solving the molecular electronic structure problem for the dihydrogen ground-state energy in the 3-21G basis described by the QCC ansatz.

In [10]:
from tangelo.algorithms.variational import VQESolver

vqe = VQESolver({"qubit_hamiltonian": qubit_ham, "ansatz": ansatz.circuit})
vqe.build()
energy_ref = vqe.simulate()
print(f"Energy from statevector simulation: {energy_ref:.4f} hartree")

Energy from statevector simulation: -1.1475 hartree


We can then refer to the optimized circuit with the `vqe.optimal_circuit` keyword and we know that with the provided Hamiltonian, the ideal energy is -1.1475 hartree.

## Visualization

- Visualization of the circuit with the qiskit module
- specify no qiskit, no viz. Try cirq.
- "Hardcode" the png

![QCC circuit for H2 in 3-21G basis](img/H2_321g_QCC.png "QCC circuit")

## Reference values

- Without Noise, tell how it should be easy to test
- Equally distributed (easiest method)
- Research is done to allocate number of measurements depending of the coefficients / variances in a qubit Hamiltonian
- We provide qubit-wise commutativity at the time of writing this notebook
- Assume number opf terms is large....

In [6]:
from tangelo.linq import Simulator
from tangelo.toolboxes.operators import QubitOperator

# qubit-wise?
n_shots_per_term = round(n_shots_budget / len(qubit_ham.terms))
shots_backend = Simulator("cirq", n_shots=n_shots_per_term, noise_model=None)
print(f"{n_shots_per_term} shots per term when equally distributed.")

6 shots per term when equally distributed.


In [7]:
energy_shots = 0.
for term, coeff in qubit_ham.terms.items():
    qubit_term = QubitOperator(term, coeff)    
    energy_shots += shots_backend.get_expectation_value(qubit_term, vqe.optimal_circuit)

print(f"Energy from equally distributed shots simulation: {energy_shots:.4f} hartree")
print(f"Error vs statevector simulation: {abs(energy_ref-energy_shots):.3f} hartree")

Energy from equally distributed shots simulation: -1.0465 hartree
Error vs statevector simulation: 0.101 hartree


## Classical Shadows

- Talk about defining the backend (without or with noise)
- Tell about the warning if n_shots =/= 1 (explain the process behind)
- Data structure and important function in our implementation.
- Scaling (in what situation good or bad)

In [8]:
cs_backend = Simulator("cirq", n_shots=1, noise_model=None)

### Randomized Single-Pauli Classical Shadows

- Summarize procedure of random sampling.

In [9]:
from tangelo.toolboxes.measurements import RandomizedClassicalShadow

random_cs = RandomizedClassicalShadow(vqe.optimal_circuit)
random_cs.build(n_shots_budget)
random_cs.simulate(cs_backend)

energy_random = random_cs.get_observable(qubit_ham)
print(f"Energy from randomized single-Pauli classical shadow: {energy_random:.4f} hartree")
print(f"Error vs statevector simulation: {abs(energy_ref-energy_random):.3f} hartree")

Energy from randomized single-Pauli classical shadow: -1.0131 hartree
Error vs statevector simulation: 0.134 hartree


### Derandomized Single-Pauli Classical Shadows

- Summarize procedure of derandom sampling.

In [10]:
from tangelo.toolboxes.measurements import DerandomizedClassicalShadow

derandom_cs = DerandomizedClassicalShadow(vqe.optimal_circuit)
derandom_cs.build(n_shots_budget, qubit_ham)
derandom_cs.simulate(cs_backend)

energy_derandom = derandom_cs.get_observable(qubit_ham)
print(f"Energy from derandomized single-Pauli classical shadow: {energy_derandom:.4f} hartree")
print(f"Error vs statevector simulation: {abs(energy_ref-energy_derandom):.3f} hartree")

Energy from derandomized single-Pauli classical shadow: -1.1578 hartree
Error vs statevector simulation: 0.010 hartree


### Adaptive Single-Pauli Classical Shadows

- Summarize procedure of adaptive sampling.

In [11]:
from tangelo.toolboxes.measurements import AdaptiveClassicalShadow

adaptive_cs = AdaptiveClassicalShadow(vqe.optimal_circuit)
adaptive_cs.build(n_shots_budget, qubit_ham)
adaptive_cs.simulate(cs_backend)

energy_adaptive = adaptive_cs.get_observable(qubit_ham)
print(f"Energy from adaptive single-Pauli classical shadow: {energy_adaptive:.4f} hartree")
print(f"Error vs statevector simulation: {abs(energy_ref-energy_adaptive):.3f} hartree")

Energy from adaptive single-Pauli classical shadow: -1.2486 hartree
Error vs statevector simulation: 0.101 hartree


## More data...

- There are some randomness in the classical shadows protocol. Therefore, your mileage may vary.
- We have ran 10 simulations, with JW and scBK encoding (8 or 6 qubits).
- Plot of abs(e_shadow - e_statevector).
![Alt Text](img/draft_classical_shadow_flavours.png "Plot")

- Discussion about the plot: classical shadow gives a probability of computing an observable within a given accuracy vs the number fo shots.
- Talk about the noise resistance -> classical shadows is more robust.

## Closing words

- Closing words...
- Summarize what has been learnt